### Collaborators of an artist

In [50]:
import pandas as pd
import itertools
from collections import defaultdict
import yaml
import glob
import os

In [52]:
# dfza = pd.read_csv("CLassification/all_files_ZAv1.csv")
# mainartists = dfza["main_artist"].unique().tolist()

In [54]:
# dfkr = pd.read_csv("CLassification/all_files_krv1.csv")
# mainartists = dfkr["main_artist"].unique().tolist()

In [56]:
dfglobal = pd.read_csv("CLassification/all_files_globalv1.csv")
mainartists = dfglobal["main_artist"].unique().tolist()

#### Functions

In [10]:
##### Remove the main artist
def theremovalofmainartist(x):
    y = list(set(x["artist_names"].split(", ")) - set([x["main_artist"]]))
    
    y = ' - '.join(map(str,y))
    
    return y

def splitanddict(x):
    art_dict = {}
    xsplit = x["artist_names"].split("-")
    xsplit = list(set(xsplit) - set(list(x["main_artist"])))
    
    art_dict[x["main_artist"]] = xsplit

    return art_dict



def flatten_list(nested_list):
    return list(itertools.chain(*nested_list))

def theconjuction(thels,x):
    xpslit = x.split("-")
    
    thels.append(xpslit)

def load_test_config(path_to_config):
    """
    Load a yaml file as the test config.
    """
    
    with open(path_to_config, 'r') as stream:
        
        parsed_yaml=yaml.safe_load(stream)
        
    return parsed_yaml

In [12]:
def countcollaboratorspermainartist(collabsdf,locale_name,mainartist):
    collabsdf["artist_names"] = collabsdf.apply(lambda x : theremovalofmainartist(x),axis=1)
    collabsdf = collabsdf[collabsdf["artist_names"] != mainartist]
    
    collabsdf.to_csv("Classification/Collaborations_data_" + locale_name + ".csv", index=False)
    
    ### Get a collaboration count for all artists
    collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
    
    ls = collabsdf[collabsdf["main_artist"]=="$"]["art_dict"].tolist()
    dd = defaultdict(list)
    
    for d in ls: # you can list as many input dicts as you want here
        for key, value in d.items():
            dd[key].append(value)
        
    # print(dd) # result: defaultdict(<type 'list'>, {1: [2, 6], 3: [4, 7]})
    
    flatone = flatten_list(list(dd.values()))
    flattwo = flatten_list(flatone)
    
    thedict = {}
    
    for aval in flattwo:
        thedict[aval]=0
    
    for aval in flattwo:
        thedict[aval]+=1
    
    ##### Select a main artist and get their collaborators
    thels = []
    thelsdict = {}
    
    thecolasdjmaphorisa = collabsdf[collabsdf["main_artist"] == mainartist]
    
    thecolasdjmaphorisa['artist_names'].apply(lambda x : theconjuction(thels,x))
    
    res_list = [y for x in thels for y in x]
    rest_list_vf = list(set(res_list))
    
    all_rest_list_vf = []
    for ares in rest_list_vf:
        theares = ares.strip()
        
        all_rest_list_vf.append(theares)
        
        all_rest_list_vf = list(set(all_rest_list_vf))
    
    thecolasdjmaphorisa = collabsdf[collabsdf["main_artist"] == mainartist]
    
    thecolasdjmaphorisa['artist_names'].apply(lambda x : theconjuction(thels,x))
    
    res_list = [y for x in thels for y in x]
    rest_list_vf = list(set(res_list))
    
    all_rest_list_vf1 = []
    for ares in rest_list_vf:
        theares = ares.strip()
        
        all_rest_list_vf1.append(theares)
        
        all_rest_list_vf1 = list(set(all_rest_list_vf1))
        
    # all_rest_list_vf1
    
    ###### Create a dictionary with 0 counts
    
    keys=all_rest_list_vf1
    values=[0]*len(all_rest_list_vf1)
    res = dict(map(lambda i,j : (i,j) , keys,values))
    
    ###### Loop through the collaborators and add to the count at every appearance count
    res_list = [y for x in thels for y in x]
    rest_list_vf = list(set(res_list))
    
    all_rest_list_vf1 = []
    for ares in rest_list_vf:
        theares = ares.strip()
        
        res[theares] +=1
        
    
    ##### write the dataframe to a csv
    thecollabsdf = pd.DataFrame.from_dict(res, orient="index")
    thecollabsdf = thecollabsdf.reset_index()

    try:
        thecollabsdf.columns = ["collab_artist", "number of times"]
        thecollabsdf=thecollabsdf.sort_values(by=["number of times"],ascending=False)
    
        thecollabsdf = thecollabsdf[thecollabsdf["collab_artist"]!=""]
    
        thecollabsdf["main_artist"] = [mainartist] * len(thecollabsdf)
        thecollabsdf.to_csv("artists_" +locale_name + "_collaborators/"+ mainartist +".csv", index=False)
    except:
        print("skipped.")
    return thecollabsdf

In [60]:
path_to_test_config1 = "configs/collabs_params.yml"
test_config1 = load_test_config(path_to_test_config1)
thelocalenames = test_config1["chartdetails"]["locales"]
# mainartists = test_config1["chartdetails"]["main_artists"]

In [62]:
for locale_name in thelocalenames:
    allfilesclass = pd.read_csv("Classification/all_files_" + locale_name + ".csv")
    collabsdf = allfilesclass[["main_artist", "artist_names"]].drop_duplicates()

    for mainartist in mainartists:
        thecollabsdf = countcollaboratorspermainartist(collabsdf, locale_name,mainartist)

C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)


skipped.


C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

skipped.


C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

...

C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

skipped.


C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

skipped.


C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collabsdf["art_dict"] = collabsdf.apply(lambda x : splitanddict(x), axis=1)
C:\Users\lkhum\AppData\Local\Temp\ipykernel_13628\4057694319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

### Get the number of artists pivotted

In [38]:
thefoldername = "artists_" +locale_name + "_collaborators/"

all_files = glob.glob(os.path.join(thefoldername, "*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f)
    all_df.append(df)
    
merged_df = pd.concat(all_df, ignore_index=True)
print(merged_df)

         collab_artist number of times main_artist
0        Ty Dolla $ign               1           $
1           Kanye West               1           $
2         Rich The Kid               1           $
3        Playboi Carti               1           $
4           LEE SUHYUN               1        10CM
...                ...             ...         ...
1093              AKMU               1      Zion.T
1094                 G               1      Zion.T
1095       Gen Hoshino               1      Zion.T
1096  Benny Benack III               1      Zion.T
1097            DRAGON               1      Zion.T

[1098 rows x 3 columns]


In [64]:
dfall = merged_df.pivot(index='main_artist', columns='collab_artist', values='number of times')
dfall = dfall.reset_index()
dfall = dfall.fillna("")

In [66]:
dfall.to_csv("Classification/"+ locale_name+"_thecollabs.csv", index=False)

In [68]:
print("done.")

done.


In [24]:
# locales: ["globalv1"]
#   main_artists : ["Ariana Grande", "Charli XCX", "KAROL G", "Taylor Swift", "Beyonc", "Selena Gomez", "SZA", "Nicki Minaj", "Shakira", "Camila Cabello", "Becky G", "Doja Cat", "Lady Gaga", "Megan Thee Stallion", "BLACKPINK"]